<a href="https://colab.research.google.com/github/PIYALI-bhunia/Multimodal_Results/blob/main/Resnet_50_Storing_Result_in_CSV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Import necessary modules

In [1]:
import os
import cv2
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
import pathlib
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten
from tensorflow.keras.optimizers import Adam, Adamax
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras import regularizers
from tensorflow.keras.models import Model, load_model
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report

In [3]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: ignored

##Load the dataset

In [4]:
data_root = pathlib.Path('/content/drive/MyDrive/Tobacco3482-jpg')

print(data_root)
for item in data_root.iterdir():
  print(item)

/content/drive/MyDrive/Tobacco3482-jpg
/content/drive/MyDrive/Tobacco3482-jpg/Letter
/content/drive/MyDrive/Tobacco3482-jpg/Tobacco3482-jpg
/content/drive/MyDrive/Tobacco3482-jpg/Scientific
/content/drive/MyDrive/Tobacco3482-jpg/Memo
/content/drive/MyDrive/Tobacco3482-jpg/Note
/content/drive/MyDrive/Tobacco3482-jpg/Report
/content/drive/MyDrive/Tobacco3482-jpg/News
/content/drive/MyDrive/Tobacco3482-jpg/Email
/content/drive/MyDrive/Tobacco3482-jpg/Form
/content/drive/MyDrive/Tobacco3482-jpg/Resume
/content/drive/MyDrive/Tobacco3482-jpg/ADVE


In [6]:
def get_file_paths_and_labels(data_root):
     img_paths = [str(path) for path in data_root.glob('*/*.jpg')]
     labels = [p.split("/")[-2] for p in img_paths]
     return img_paths, labels

img_paths, labels = get_file_paths_and_labels(data_root)
# print(img_paths)
# print(labels)
print(len(img_paths))
print(len(labels))

3482
3482


##Create the dataframe

In [7]:
df = pd.DataFrame(list(zip(img_paths, labels)),
               columns =['image_path', 'data_label'])
df.head()

,image_path,data_label
0,/content/drive/MyDrive/Tobacco3482-jpg/Letter/...,Letter
1,/content/drive/MyDrive/Tobacco3482-jpg/Letter/...,Letter
2,/content/drive/MyDrive/Tobacco3482-jpg/Letter/...,Letter
3,/content/drive/MyDrive/Tobacco3482-jpg/Letter/...,Letter
4,/content/drive/MyDrive/Tobacco3482-jpg/Letter/...,Letter


In [8]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
df['data_label']= le.fit_transform(df['data_label'])
df.head()

,image_path,data_label
0,/content/drive/MyDrive/Tobacco3482-jpg/Letter/...,3
1,/content/drive/MyDrive/Tobacco3482-jpg/Letter/...,3
2,/content/drive/MyDrive/Tobacco3482-jpg/Letter/...,3
3,/content/drive/MyDrive/Tobacco3482-jpg/Letter/...,3
4,/content/drive/MyDrive/Tobacco3482-jpg/Letter/...,3


In [9]:
df['data_label'] = [str(i) for i in df['data_label']]

In [10]:
labels_dict = {}
for idx, this_path in enumerate(df['image_path']):
    labels_dict[this_path.split("/")[-2]] = df['data_label'][idx]
    
print(labels_dict)

{'Letter': '3', 'Scientific': '9', 'Memo': '4', 'Note': '6', 'Report': '7', 'News': '5', 'Email': '1', 'Form': '2', 'Resume': '8', 'ADVE': '0'}


In [11]:
balance=df['data_label'].value_counts()
print (balance)

4    620
1    599
3    567
2    431
7    265
9    261
0    230
6    201
5    188
8    120
Name: data_label, dtype: int64


##Split the dataframe into train , test and validation

In [12]:
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
import random
random.seed(56)

In [ ]:
batch_size=32
epochs = 30
fine_tune_epochs = 20
def scalar(x):
    return x/127.5-1

d = {
    "precision" : [],
     "recall" : [],
     "f1_score" : [],
     "accuracy" : [],
     "seed_value" : []
}
seed = []


for i in range(5):
    print('#Run: ', i+1)
    sd = random.randint(0, 100)
    seed.append(sd)

    train_df, dummy_df=train_test_split(df, test_size=0.3, shuffle=True, random_state=0)
    test_df, valid_df= train_test_split(dummy_df, test_size=0.5, shuffle=True, random_state=0)

    length=len(test_df)

    ######## Keras datagenerator #########
    trgen=tf.keras.preprocessing.image.ImageDataGenerator(preprocessing_function=scalar)

    train_gen=trgen.flow_from_dataframe(train_df, x_col='image_path', y_col='data_label', target_size=(224,224), class_mode='categorical',batch_size=batch_size, shuffle=True, seed=sd)

    tvgen=tf.keras.preprocessing.image.ImageDataGenerator(preprocessing_function=scalar)

    valid_gen=tvgen.flow_from_dataframe(valid_df, x_col='image_path', y_col='data_label', target_size=(224,224), class_mode='categorical',batch_size=batch_size, shuffle=False, seed=sd)
   
    # determine test generator batch size and steps to go through the test set exactly once for predictions
    test_batch_size=sorted([int(length/n) for n in range(1,length+1) if length % n ==0 and length/n<=batch_size],reverse=True)[0]  

    test_steps=int(length/test_batch_size)

    test_gen=tvgen.flow_from_dataframe(test_df, x_col='image_path', y_col='data_label', target_size=(224,224), class_mode='categorical',batch_size=test_batch_size, shuffle=False, seed=sd)

    test_labels=test_gen.labels
    

    img_shape=(224,224,3)
    class_count = 10
    dropout=0.2
    lr= 0.001
    base_model= tf.keras.applications.ResNet50(include_top=False,
                   input_shape=(224,224,3),
                   pooling='avg',classes=10,
                   weights='imagenet')

    base_model.trainable=False

    inputs = keras.Input(shape=img_shape)
    x = base_model(inputs, training=False)

    # x=tf.keras.layers.Dropout(rate=dropout, seed=seed_value)(x)
    x =tf.keras.layers.Dense(2048, kernel_regularizer = regularizers.l2(l = 0.016),activity_regularizer=regularizers.l1(0.006), bias_regularizer=regularizers.l1(0.006) ,activation='relu', kernel_initializer= tf.keras.initializers.GlorotUniform(seed=sd))(x)
    # x=tf.keras.layers.Dropout(rate=dropout, seed=seed_value)(x)
    outputs=tf.keras.layers.Dense(class_count, activation='softmax',kernel_initializer=tf.keras.initializers.GlorotUniform(seed=sd))(x)
    model=Model(inputs, outputs)

    model.compile(Adamax(learning_rate=lr), loss='categorical_crossentropy', metrics=['accuracy'])

    from keras.callbacks import ModelCheckpoint
    checkpoint = ModelCheckpoint("/model-{epoch:03d}-{accuracy:03f}-{val_accuracy:03f}.h5", verbose=1, monitor='val_loss',save_best_only=True, mode='auto') 

    history=model.fit(x=train_gen,  epochs=epochs, verbose=2, validation_data=valid_gen, callbacks=[checkpoint],
               validation_steps=None,  shuffle=False,  initial_epoch=0)
    

     ####### Fine-Tune ######
    base_model.trainable=True
    tune_epochs = fine_tune_epochs
    total_epochs= epochs + tune_epochs

    model.compile(Adamax(learning_rate=lr/10), loss='categorical_crossentropy', metrics=['accuracy'])
    
    history_fine = model.fit(x=train_gen,  epochs=total_epochs, verbose=2, validation_data=valid_gen,
               validation_steps=None,  shuffle=False,  initial_epoch=epochs)
    

    y_pred = model.predict(test_gen)
    y_pred = np.argmax(y_pred, axis=1)

    y_true = test_gen.classes

    d["precision"].append(float(precision_score(y_true, y_pred, average='weighted')))
    d["recall"].append(float(recall_score(y_true, y_pred, average='weighted')))
    d["f1_score"].append(float(f1_score(y_true, y_pred, average='weighted')))
    d['accuracy'].append(float(accuracy_score(y_true, y_pred)))

#Run:  1
Found 2437 validated image filenames belonging to 10 classes.
Found 523 validated image filenames belonging to 10 classes.
Found 522 validated image filenames belonging to 10 classes.
94781440/94765736 [==============================] - 1s 0us/step
Epoch 1/30

Epoch 1: val_loss improved from inf to 3.17281, saving model to /model-001-0.378334-0.464627.h5
77/77 - 794s - loss: 9.4143 - accuracy: 0.3783 - val_loss: 3.1728 - val_accuracy: 0.4646 - 794s/epoch - 10s/step
Epoch 2/30

Epoch 2: val_loss improved from 3.17281 to 2.48442, saving model to /model-002-0.459171-0.447419.h5
77/77 - 476s - loss: 2.7484 - accuracy: 0.4592 - val_loss: 2.4844 - val_accuracy: 0.4474 - 476s/epoch - 6s/step
Epoch 3/30

Epoch 3: val_loss improved from 2.48442 to 2.21101, saving model to /model-003-0.479688-0.483748.h5
77/77 - 456s - loss: 2.3180 - accuracy: 0.4797 - val_loss: 2.2110 - val_accuracy: 0.4837 - 456s/epoch - 6s/step
Epoch 4/30

Epoch 4: val_loss improved from 2.21101 to 2.07047, saving mo

In [ ]:
from google.colab import files
print(seed)
d['seed_value'] = seed
result = pd.DataFrame(d, index = ["run-"+str(i+1) for i in range(5)])
print(result)
result.to_csv("Resnet50.csv")
files.download('Resnet50_result.csv')